In [416]:
import scipy
from scipy.io.wavfile import read
from scipy.signal import hann
from scipy.fftpack import rfft
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats

In [439]:
audios=['H_A00226225.wav','H_A00226522.wav','H_A00226860.wav',
       'H_A01227071.wav','H_A01228574.wav','M_A00344189.wav',
       'H_A01229141.wav','H_A01630076 .wav','M_A01226436.wav']

In [440]:
num_segments = 4

In [441]:
audio_data = {'db_mean': [], 'db_min': [], 'db_median': [], 'label': []}

In [442]:
for audio_file in audios:
    input_data = read(audio_file)
    audio = input_data[1]
    a_l = len(audio)
    # apply a Hanning window
    window = hann(a_l)
    newAudio = audio[:a_l,0] * window
    # fftz
    mags = abs(rfft(newAudio))
    # convert to dB
    mags = 20 * scipy.log10(mags)
    # normalise to 0 dB max
    mags -= max(mags)
    audio_data['db_mean'].append(mags.mean())
    audio_data['db_min'].append(mags.min())
    audio_data['db_median'].append(np.median(mags))
    audio_data['label'].append('Macho' if audio_file[0] == 'H' else 'Hembra')
audio_df = pd.DataFrame(audio_data)
audio_df

,db_mean,db_min,db_median,label
0,-80.950480,-206.273642,-76.653750,Macho
1,-67.043662,-187.411110,-67.692895,Macho
2,-56.537304,-167.733353,-52.810290,Macho
3,-67.908956,-196.834957,-67.900460,Macho
4,-89.841875,-206.577406,-87.643456,Macho
5,-70.518264,-211.278746,-68.381244,Hembra
6,-79.334454,-244.485079,-71.637910,Macho
7,-99.601006,-230.553409,-113.387565,Macho
8,-86.640875,-243.925711,-99.571895,Hembra


## Machine Learning classifier

In [443]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

In [444]:
X = df.drop('label',axis=1)
y = df['label']

In [445]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [446]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [447]:
rfc_pred = rfc.predict(X_test)

In [448]:
X_test

,db_mean,db_min,db_max
2,-56.537304,-167.733353,0.0
1,-67.043662,-187.411110,0.0


In [449]:
print(confusion_matrix(y_test,rfc_pred))

[[2]]


In [450]:
print(classification_report(y_test,rfc_pred))

              precision    recall  f1-score   support

       Macho       1.00      1.00      1.00         2

   micro avg       1.00      1.00      1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

